<a href="https://colab.research.google.com/github/souissiaya/Facial-Emotions-Detection/blob/main/FacialEmotionsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install libraries
!pip install opencv-python
!pip install matplotlib
!pip install tensorflow

In [ ]:
#Install library
!pip install keras_preprocessing

In [ ]:
#Import Required Packages
import tensorflow as tf
import cv2

import os
import matplotlib.pyplot as plt
import numpy as np

from keras_preprocessing.image import load_img, img_to_array



In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
#copy the kaggle.jsom to the folder created
! cp kaggle.json ~/.kaggle/
#Permission for the json to act
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d msambare/fer2013

In [ ]:
!unzip fer2013.zip


**Image count for each class**


In [ ]:
# Define name of each classes (all image folder names)
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral','sad', 'surprise']

# Print number of images for each class
folder_path = "data/"
for cls in classes:
    path = os.path.join(folder_path, '/content/train', cls)
    lst = os.listdir(path)
    number_files = len(lst)
    print(cls, ': ', number_files)

**Create a folder structure using Python**


In [ ]:
# Create a empty folder
os.makedirs('new_data')
# Create "train" folder inside new_data folder
os.makedirs('new_data/train')
# Crate sub-folders
for cls in classes:
    os.makedirs('new_data/train/'+cls)

**Copy images to Subfolder**


In [ ]:
# Copy 436 files to new folder
import shutil
import random

num_files = 436

for cls in classes:
    # Downloaded original training image folder path for face emotion recognition
    src_path = os.path.join('/content', 'train', cls)
    # Sub folder path
    dst_path = os.path.join('/content/new_data', 'train', cls)
    src_files = os.listdir(src_path)
    # Select random 436 images from source directory
    src_select_files = random.sample(src_files, num_files)

    # Copy selected images to destination folder
    for file_name in src_select_files:
        full_file_name = os.path.join(src_path, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, dst_path)


In [ ]:
num_files = 436

for cls in classes:
    # Downloaded original training image folder path for face emotion recognition
    src_path = os.path.join('/content', 'train', cls)
    # Sub folder path
    dst_path = os.path.join('/content/new_data', 'train', cls)
    src_files = os.listdir(src_path)
    # Select random 436 images from source directory
    src_select_files = random.sample(src_files, num_files)

    # Copy selected images to destination folder
    for file_name in src_select_files:
        full_file_name = os.path.join(src_path, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, dst_path)


In [ ]:
# Print number of images for each class
folder_path = "new_data/"
for cls in classes:
    path = os.path.join(folder_path, 'train', cls)
    lst = os.listdir(path)
    number_files = len(lst)
    print(cls, ': ', number_files)


**Show images using OpenCV**


In [ ]:
picture_size = 48
folder_path = "new_data/"

expression = 'disgust'

plt.figure(figsize= (12,12))
for i in range(1, 10, 1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"train/"+expression+"/"+
                  os.listdir(folder_path + "train/" + expression)[i], target_size=(picture_size, picture_size))
    plt.imshow(img)
plt.show()

**Change Image size**


In [ ]:
# Function to Read all the images: resize and convert in them to array using opencv

img_size = 224 ## ImageNet => 224x224
training_data = []

for category in classes:
  path = os.path.join(folder_path, 'train', category)
  class_num = classes.index(category)
  for img in os.listdir(path):
      try:
          img_array = cv2.imread(os.path.join(path, img))
          new_array = cv2.resize(img_array, (img_size, img_size))
          training_data.append([new_array, class_num])
      except Exception as e:
          pass


In [ ]:
# Check the training data shape
temp_array = np.array(training_data)
temp_array.shape

In [ ]:
# The shape of one converted image
# Reading one image from angry folder
img_array = cv2.imread('/content/new_data/train/disgust/Training_11660541.jpg')
print('Input image shape: ', img_array.shape)

# Convert image to 224x224
img_size = 224 ## ImageNet => 224x224
new_array = cv2.resize(img_array, (img_size, img_size))
print('Converted image shape: ', new_array.shape)

**Add image dimensions**


In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3) # converting it to 4 dimention

print(X.shape)

# Convert to array
Y = np.array(y)

**Setup MobileNet model**


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Download Pre-trained MobileNet Model
model = tf.keras.applications.MobileNetV2() ## Pre-trained Model

# Print MobileNet architecture
model.summary()

**Note:** In the model architecture, if you see the last layer has 1000 classes. But for our case, we have only 7 classes (angry, disgust, fear, happy, neutral, sad, and surprise). So we need to change the last layer of the downloaded MobileNet pre-trained model. This technique is called Transfer Learning.

In [ ]:
# Defining first layer as input layer of Mobilenet
base_input = model.layers[0].input
print(base_input)

# Removing last layer of MobileNet model
base_output = model.layers[-2].output

# Adding some extra layers
final_output = layers.Dense(128)(base_output) ## adding new layer, after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) ## activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
# Defining final layer with 7 classes
final_output = layers.Dense(7, activation = 'softmax')(final_output) ## 7 because my classes are 7


**Note:** Here in this code we are removing the last layer from the downloaded model and adding our customized output layer with some additional layers.

In [ ]:
custom_model = keras.Model(inputs = base_input, outputs = final_output) ## Final model architecture
# Print our custom model summary
custom_model.summary()

# Compiling the model to train
custom_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


For this example, I am using:
  - Sparse categorical cross entropy as loss function
  - adam as optimizer


Now, I am going to use batch size = 8 and 25 epochs to avoid any memory error while training in the local windows system.

In [ ]:
custom_model.fit(X, Y, epochs = 80, batch_size = 8)

The final accuracy of our custom emotion detection model is 98%

**Save trained model**


In [ ]:
custom_model.save('facial_expression_model.h5')


We are done with our custom model training for emotion detection. Now let’s test how our model is performing for any image.

In [ ]:
# Read downloaded test image in Opencv
test_img = cv2.imread('/content/test/neutral/PrivateTest_11752870.jpg')
# Take a backup of input image before face detection
img_bcp = test_img.copy()

# Show image in OpenCV
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))


**Face detection and cropping**


To detect facial expressions we first need to detect the face.

In [ ]:
#Define haar cascade classifier for face detection
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Convert image to gray scale OpenCV
gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

# Detect face using haar cascade classifier
faces_coordinates = face_classifier.detectMultiScale(gray_img)

# Draw a rectangle around the faces
for (x, y, w, h) in faces_coordinates:
    # Draw rectangle around face
    cv2.rectangle(test_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Crop face from image
    cropped_face = img_bcp[y:y+h, x:x+w]

# Plot original image
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))

# Plot cropped image after performing face detection
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB))



**Model Prediction**


In [ ]:
#  Creating class dictionary
class_dictionary = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

final_image = cv2.resize(test_img, (224,224))
final_image = np.expand_dims(final_image, axis=0) ## Need 4th dimension
final_image = final_image/255.0 ## Normalizing

# Load model
new_model = tf.keras.models.load_model('/content/facial_expression_model.h5')
prediction = new_model.predict(final_image)
class_dictionary[np.argmax(prediction)]